In [1]:
# This step is optional if you environment is not trusted by your Python host machine
import os
os.environ['CAS_CLIENT_SSL_CA_LIST'] = 'C:/Users/gerdaw/Documents/SANDBOX/trustedcerts.pem'

In [ ]:
import swat

In [3]:
# Ensuring that you have the minimum version of SWAT to have access to the two new functions
if swat.__version__ >= '1.13.1':
    print('Perfect you are all set to go ahead with this example')
else:
    print('Please use pip install swat --upgrade to update the SWAT version')

Perfect you are all set to go ahead with this example


In [4]:
# Get information to authenticate with SAS Viya
VIYAHOST = input('Enter your SAS Viya Host URL: ')
AUTHCODE = input(f'Open the following [link]({VIYAHOST}/SASLogon/oauth/authorize?client_id=SWAT&response_type=code) and copy the Authcode: ')

In [5]:
# Connect to SAS Viya via the HTTPS method not binary
conn = swat.CAS(f'{VIYAHOST}/cas-shared-default-http', 443, authcode=AUTHCODE)
conn.serverstatus().get_tables('nodestatus')[0]

NOTE: Grid node action status report: 5 nodes, 9 total actions executed.


,name,role,uptime,running,stalled
0,worker-3.sas-cas-server-default.sasviya4.svc.c...,worker,2.597,0,0
1,worker-2.sas-cas-server-default.sasviya4.svc.c...,worker,2.597,0,0
2,worker-0.sas-cas-server-default.sasviya4.svc.c...,worker,2.596,0,0
3,worker-1.sas-cas-server-default.sasviya4.svc.c...,worker,2.596,0,0
4,controller.sas-cas-server-default.sasviya4.svc...,controller,2.658,0,0


In [6]:
# Upload HMEQ to CAS
conn.upload('https://raw.githubusercontent.com/sassoftware/model-management-resources/main/samples/Data/hmeq.csv')
hmeq = conn.CASTable('hmeq', caslib='casuser')

NOTE: Cloud Analytic Services made the uploaded file available as table HMEQ in caslib CASUSER(gerdaw).
NOTE: The table HMEQ has been created in caslib CASUSER(gerdaw) from binary data uploaded to Cloud Analytic Services.


In [7]:
# Display the first five rows
hmeq.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1.0,1100.0,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1.0,1300.0,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1.0,1500.0,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1.0,1500.0,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1700.0,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN


## New CASTable method nunique

The new method _nunique_ for CASTable objects returns the unqiue elements for each column as a list.

In [8]:
uniqueElements = hmeq.nunique()
print(uniqueElements)

BAD           2
LOAN        540
MORTDUE    5053
VALUE      5381
REASON        2
JOB           6
YOJ          99
DEROG        11
DELINQ       14
CLAGE      5314
NINQ         16
CLNO         62
DEBTINC    4693
dtype: int64


## New CASTable method drop_duplicates

The new method drop_duplicates for CASTable objects retuns a CASTable which contains no more duplicate elements.

You have to specify the attribute _casout_ to specify where the result should be stored. Optionally you can also specify a string/list of all the columns that should be considered for deduplication.

In [9]:
dropedElements = hmeq.drop_duplicates(casout='hmeqNoDubs', subset='JOB')

In [10]:
print(f'The CASTable after dropping duplicates now has {dropedElements.shape[0]} rows.')
print(f'The original HMEQ CASTable had {hmeq.shape[0]} rows.')

The CASTable after dropping duplicates now has 7 rows.
The original HMEQ CASTable had 5960 rows.
